In [16]:
import functools
import numpy as np
import pandas as pd
import typing

from tokenizers.processors import TemplateProcessing
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer
from datasets import list_datasets, load_dataset
from datasets import Dataset

In [17]:
corpus_excel_path = '/home/USER/Documents/Uni/WiSe2223/Consulting/mlw-consulting-project/data/raw/Lemmaliste.xlsx'
corpus_data_json_path = '/home/USER/Documents/Uni/WiSe2223/Consulting/mlw-consulting-project/data.json'
corpus = pd.read_excel(corpus_excel_path)['Lemmata'].to_frame()
# corpus = corpus.dropna()
corpus_data_json = pd.read_json(corpus_data_json_path)
# corpus_data_json = corpus_data_json.dropna()
corpus_data_json = corpus_data_json.rename(columns={'lemma': 'Lemmata'})
corpus['Lemmata'] = corpus['Lemmata'].astype(str)
corpus_data_json['Lemmata'] = corpus_data_json['Lemmata'].astype(str)
corpus_data_json = corpus_data_json.drop("id", axis=1)
corpus_data_json = corpus_data_json.drop_duplicates()
# corpus.join(corpus_data_json, on = "Lemmata", how = "inner")
corpus = pd.concat([corpus, corpus_data_json])

rm_list: list = []
for i, e in enumerate(corpus['Lemmata'].values):
    if isinstance(e, float):
        rm_list.append(e)
corpus = corpus[~corpus['Lemmata'].isin(rm_list)]
corpus.head(5)


,Lemmata
0,acarus
1,a
2,aano
3,aardum
4,aardus


In [18]:
dataset = Dataset.from_pandas(corpus)
dataset

Dataset({
    features: ['Lemmata', '__index_level_0__'],
    num_rows: 46979
})

In [19]:
# I use tutorial code from https://huggingface.co/docs/tokenizers/quicktour as example
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
trainer = BpeTrainer(vocab_size=50, special_tokens=["[BOS]", "[UNK]", "[EOS]"]) #  Adding [BOS] and [EOS] here
tokenizer.pre_tokenizer = Whitespace()

# Please using TemplateProcessing
# https://huggingface.co/docs/tokenizers/api/post-processors#tokenizers.processors.TemplateProcessing
from tokenizers.processors import TemplateProcessing
tokenizer.post_processor = TemplateProcessing(
        single="[BOS] $A [EOS]",
        special_tokens=[("[BOS]", 0 ), ("[EOS]", 2)],
)

##################################################

files = ["vocab.txt"]
def batch_iterator():
    for i in range(0, len(dataset), 128):
        yield dataset[i : i + 128]["Lemmata"]
tokenizer.train_from_iterator(
        batch_iterator(),
        trainer=trainer)

output = tokenizer.encode("gardo")
print(output.ids)
# >> [1, 27255, 18, 95, 13, 5099, 7, 7963, 5114, 6220, 0, 37, 2] <-- you can see there are token [1] in the begining and token [2] at the end of the sequence
print(tokenizer.decode(output.ids, skip_special_tokens=False))
# >> no [BOS] and [EOS] after decoding

tokenizer.save("byte-level-BPE.tokenizerTEST.json")





[0, 32, 26, 43, 29, 40, 2]
[BOS] g a r d o [EOS]


In [25]:
tokenizer.add_special_tokens(["[PAD]"])

1

In [ ]:
special_tokens_map = {'cls_token': '<CLS>', 'pad_token':'<PAD>', 'sep_token': '<SEP>', 'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<UNK>', 'additional_special_tokens': '@'}
num_added_toks = tokenizer.add_special_tokens(list(special_tokens_map))

In [28]:
special_tokens_dict = {
    'pad_token': '<PAD>',
    'cls_token': '<CLS>',
    'bos_token': '[BOS]',
    'eos_token': '<|endoftext|>',
    'unk_token': '<UNK>'}
tokenizer.add_special_tokens(special_tokens_dict)

TypeError: argument 'tokens': 'dict' object cannot be converted to 'PyList'

In [ ]:
len(tokenizer.get_vocab())

52